Testing seawater for TEOS-10 compatibility

In [1]:
import gsw
import numpy as np
import scipy.io

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


# Matlab tests

In [2]:
# Load gsw Matlab demo file
f = scipy.io.loadmat('/data/nsoontie/GSW/library/gsw_data_v3_0.mat')
demo = f['gsw_demo_data']
# grab data - subsampled like in matlab demo. Why is creating a list of indices so hard?
ind_sub = np.arange(28,48,4)
ind_sub =  [i for i in ind_sub]
indices = [0,21]
for i in ind_sub:
    indices.append(i)

p = np.array([demo['p'][0][0][i,0] for i in indices])
t = np.array([demo['t'][0][0][i,0] for i in indices])
SP = np.array([demo['SP'][0][0][i,0] for i in indices])


long = demo['long'][0][0][0,0]
lat = demo['lat'][0][0][0,0]

## Practical salinity to absolute salinity

In [3]:
#copied from Matlab gsw_demo output
SA_Mat = np.array([ 34.7392,  34.4652,  34.7738,  34.8435,  34.8637,  34.8739,  34.8776,] )

In [4]:
SA_py = gsw.SA_from_SP(SP,p, long, lat)
SA_py

array([ 34.73922653,  34.46519542,  34.7738161 ,  34.84347021,
        34.86368827,  34.87389077,  34.87760285])

In [5]:
(np.abs(SA_py-SA_Mat) < 1e-4).all()

True

Yes, up to the accuracy I copied from Matalb results, the answers are the same.

## In-situ temperature to conservative temperature

In [6]:
#copied from output of gsw_demo
CT_Mat = np.array([ 19.5130,   3.5588,   1.8157,   1.3434,   1.1583,   1.0518,  1.0073, ] )

CT_py = gsw.CT_from_t(SA_py, t, p)
CT_py

array([ 19.51302027,   3.55877109,   1.81571146,   1.34340125,
         1.15831711,   1.05175362,   1.0073248 ])

In [7]:
(np.abs(CT_py - CT_Mat) < 1e-4).all()

True

Again, up to accuracy copied from matlab, the results are the same.

## Density

In [8]:
#copied from output of gsw_demo
rho_Mat = np.array([ 1024.5709,  1031.9377,  1037.0031,  1041.6695,  1046.1799,  1050.5915, 1054.9014, ] )

rho_py = gsw.rho(SA_py, CT_py, p)
rho_py

array([ 1024.57149173,  1031.93720792,  1037.00233755,  1041.6688784 ,
        1046.17966141,  1050.59166961,  1054.90183236])

In [9]:
(np.abs(rho_py-rho_Mat) < 1e-4).all()

False

In [10]:
rho_py-rho_Mat

array([ 0.00059173, -0.00049208, -0.00076245, -0.0006216 , -0.00023859,
        0.00016961,  0.00043236])

Ok, I'm starting to get some differences here. They are pretty small (< 1e-3)

# Notes

### What is conservative temperature?
